In [ ]:
from frontend.database import session
from scripts import generate_fc_layer_circuit, benchmark_utils
import numpy as np

In [ ]:
weight_size = 2000
weight = np.asarray([int(np.random.random() > 0.5) for i in range(weight_size)])

In [ ]:
circuit_file=generate_fc_layer_circuit.generate(weight.size, weight)
print(circuit_file)

In [ ]:
inp_var = benchmark_utils.get_inputs(circuit_file)
out_var = benchmark_utils.get_outputs(circuit_file)
inp_dict = {}
for inps in inp_var:
    inp_dict[inps] = int(np.random.random() > 0.5)

In [ ]:
inputs_file = benchmark_utils.write_inputs_file(inp_dict)

In [ ]:
results = benchmark_utils.run_circuit(circuit_file,inputs_file,"bool","TFHE")

In [ ]:
assert np.alltrue([int(weight[i]==inp_dict['FCINPUT_'+str(i)]) == int(results['Outputs']['XNOROUTPUT_'+str(i)]) for i in range(weight_size)])

In [ ]:
print(results['Processing times (s)']['circuit_evaluation'])

In [ ]:
from scripts.circuit import circuit
from scripts.particles import variables, enc_vec
from scripts.reusable_modules import oneb_adder, nb_adder

In [ ]:
x = variables('x')
y = variables('y')
cin = variables('cin')
s = variables('sum')
c = variables('carry')
b1_adder = oneb_adder('1b_adder', [x, y, cin], [s, c], 1)
c_obj = circuit('1 Bit Adder', b1_adder)
c_obj.write_file(filename='./test.sheep')
inp_dict = {'x':1, 'y':1, 'cin':1}
inputs_file = benchmark_utils.write_inputs_file(inp_dict)
results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")

In [ ]:
print('Carry : {:d} and Sum : {:d}'.format(int(results['Outputs']['carry']),int(results['Outputs']['sum'])))

In [ ]:
x1 = variables('x1')
x2 = variables('x2')
x3 = variables('x3')
y1 = variables('y1')
y2 = variables('y2')
y3 = variables('y3')
cin = variables('cin')
s1 = variables('sum1')
s2 = variables('sum2')
s3 = variables('sum3')
s4 = variables('sum4')
b3_adder = nb_adder('2b_adder', [cin, x1, x2, x3, y1, y2, y3], [s1, s2, s3, s4], 3)
c_obj = circuit('3 Bit Adder', b3_adder)
c_obj.write_file(filename='./test.sheep')
inp_dict = {'x1':0, 'x2':1, 'x3': 1, 'y1':1, 'y2':1,'y3':1,'cin':0}
inputs_file = benchmark_utils.write_inputs_file(inp_dict)
results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")

In [ ]:
print('{:d}{:d}{:d}{:d}'.format(int(results['Outputs']['sum1']),
                            int(results['Outputs']['sum2']), 
                            int(results['Outputs']['sum3']), 
                            int(results['Outputs']['sum4'])
                           ))

In [ ]:
cin = variables(name='cin', randomize_name=4)
t1 = enc_vec(name='v1', nb=3, randomize_name=4)
t2 = enc_vec(name='v2', nb=3, randomize_name=4)
sum_vec = enc_vec(name='sum', nb=4, randomize_name=4)

In [ ]:
t3 =  t1 + t2
print t3.nb, len(t3)
t3.push(cin)

In [ ]:
b3_adder = nb_adder('2b_adder', t3, sum_vec, 3)
c_obj = circuit('2 Bit Adder', b3_adder)
c_obj.write_file(filename='./test.sheep')

In [ ]:
cin_inp = [1]
t1_inp = [1, 1, 1]
t2_inp = [1, 1, 1]

In [13]:
inp_dict = t3.get_input_dict(cin_inp+t1_inp+t2_inp)
inputs_file = benchmark_utils.write_inputs_file(inp_dict)
results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")
out_vars = sum_vec.get_variables()
print ''.join(results['Outputs'][var] for var in out_vars)

NameError: name 't3' is not defined

In [54]:
from scripts.functions import reduce_add
inp = enc_vec(name='in', nb=8)
print inp.get_variables()
out = enc_vec(name='out', nb=4)
ra = reduce_add('adder', inp, out)
ra_obj = circuit('reduce_add', ra)
ra_obj.write_file(filename='./test.sheep')

['in0D767', 'in1CB94', 'in2C70B', 'in31F80', 'in4B960', 'in5D170', 'in6B743', 'in7A177']
0
['in0D767', 'in1CB94']
1
['in2C70B', 'in31F80']
2
['in4B960', 'in5D170']
3
['in6B743', 'in7A177']
0
['adder_ra_105962', 'adder_ra_1172FA', 'adder_ra_1238C4', 'adder_ra_132EF0']
1
['adder_ra_141B20', 'adder_ra_15F4EC', 'adder_ra_16C083', 'adder_ra_1793A9']
0
['adder_ra_20DC2D', 'adder_ra_2144D3', 'adder_ra_22402B', 'adder_ra_23E8A8', 'adder_ra_248C66', 'adder_ra_25E90A']


In [55]:
print(inp.get_variables())
print(ra)
print(out.get_variables())

['in0D767', 'in1CB94', 'in2C70B', 'in31F80', 'in4B960', 'in5D170', 'in6B743', 'in7A177']
in0D767 in1CB94 ADD 2b_adder_0_tmp13
2b_adder_0_tmp13 ALIAS adder_ra_105962
in0D767 in1CB94 MULTIPLY 2b_adder_0_tmp24
2b_adder_0_tmp24 ALIAS adder_ra_1_adder_0_carry_14
adder_ra_1_adder_0_carry_14 ALIAS adder_ra_1172FA
in2C70B in31F80 ADD 2b_adder_0_tmp13
2b_adder_0_tmp13 ALIAS adder_ra_1238C4
in2C70B in31F80 MULTIPLY 2b_adder_0_tmp26
2b_adder_0_tmp26 ALIAS adder_ra_1_adder_1_carry_1B
adder_ra_1_adder_1_carry_1B ALIAS adder_ra_132EF0
in4B960 in5D170 ADD 2b_adder_0_tmp18
2b_adder_0_tmp18 ALIAS adder_ra_141B20
in4B960 in5D170 MULTIPLY 2b_adder_0_tmp29
2b_adder_0_tmp29 ALIAS adder_ra_1_adder_2_carry_1C
adder_ra_1_adder_2_carry_1C ALIAS adder_ra_15F4EC
in6B743 in7A177 ADD 2b_adder_0_tmp15
2b_adder_0_tmp15 ALIAS adder_ra_16C083
in6B743 in7A177 MULTIPLY 2b_adder_0_tmp22
2b_adder_0_tmp22 ALIAS adder_ra_1_adder_3_carry_1B
adder_ra_1_adder_3_carry_1B ALIAS adder_ra_1793A9
adder_ra_105962 adder_ra_1238C4 ADD

In [56]:
inp_arr_val = [0, 0,0, 0, 0, 0,0, 0]
for idx in range(0, 1):
    inp_dict = inp.get_input_dict(inp_arr_val)
    inputs_file = benchmark_utils.write_inputs_file(inp_dict)
    results = benchmark_utils.run_circuit('./test.sheep',inputs_file,"bool","TFHE")
    out_vars = out.get_variables()
    err_str = ''.join(results['Outputs'][var] for var in out_vars)+" --" +bin(np.sum(inp_arr_val))[::-1]
    assert ''.join(results['Outputs'][var] for var in out_vars) == bin(np.sum(inp_arr_val))[::-1], err_str

AssertionError: 0000 --0b0